In [3]:
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd # this module is useful to work with tabular data
import random # this module will be used to select random samples from a collection
import os # this module will be used just to create directories in the local filesystem
from tqdm.notebook import tqdm # this module is useful to plot progress bars

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torch import nn

## Import data

In [ ]:
particle_df_path = '../data/particle_df.csv'
particle_preproc_df_path = '../data/particle_df_preprocessed.csv'
par_pre_df = pd.read_csv(particle_preproc_df_path)
par_df = pd.read_csv(particle_df_path)
par_df.head()

## Class Dataset

In [ ]:
class ParticleDataset(Dataset):
    # The ParticleDataset class inherits the Dataset class and implements the __init__, __len__, and __getitem__ methods

    def __init__(self, path, transform=None):
        # Initializing the ParticleDataset object.
        # "path" is the path to the csv file containing the particle data.
        # "transform" is an optional argument that specifies the transformations to be applied to the data.

        # Read the csv file into a Pandas DataFrame.
        self.x = pd.read_csv(path)

        # Put the coordinates eta and phi as the first two features
        self.x = self.x.reindex(
            columns=["particlePolarPy", "particlePhi"]
            + [
                col
                for col in self.x.columns
                if col not in ["particlePolarPy", "particlePhi"]
            ]
        )

        # Store the "transform" argument.
        self.transform = transform

    def __len__(self):
        """
        Returns the number of particles in the dataset.
        """
        # Return the number of rows in the DataFrame (i.e., the number of particles).
        return self.x.shape[0]

    def __getitem__(self, idx):
        """
        Returns the particles with jetID = idx.
        """
        # Get the rows in the DataFrame that have a "jetID" column equal to "idx".
        x = self.x[self.x.jetID==idx].to_numpy()
        
        # If "transform" was specified, apply it to the data.
        if self.transform:
            x = self.transform(x)
        
        # Return the transformed data.
        return x

### Transforms

In [ ]:
# Create a Compose object that applies the "ToTensor" transformation.
train_transform = transforms.Compose([
    transforms.ToTensor(),
])


# Create a ParticleDataset object using the csv file located at "particle_df_path" and the "train_transform" transformations.
train_data = ParticleDataset(particle_df_path, train_transform)

# Access the first element in the dataset to get its shape.
train_data[1].shape

## Dataloader

Custom collate only for a dataset with jets composed by a different number of particles

In [ ]:
def custom_collate(batch):
    """
    A custom collate function that can handle different shape tensors.
    The default collate function provided by PyTorch's DataLoader assumes that all tensors in a batch have the same shape. 
    However, in our case, each "datum" is a set of particles that compose a jet and the number of particles composing a jet is not fixed. 
    Therefore, each tensor representing a jet has a different shape.

    To handle this scenario, we need to override the collate function to be able to stack the tensors into a batch. 
    This function first determines the maximum number of particles among all jets in the batch. 
    Then, it pads all tensors with zeros to make sure they have the same shape. 
    Finally, it stacks the tensors along the batch dimension to return the padded data and original lengths.

    """
    
    # Get the max number of particles among all the jets in the batch
    n_part_max = max(x.shape[1] for x in batch)

    # Pad all the tensors with zeros so they have the same shape
    data = []
    lengths = []
    for x in batch:
        n_part = x.shape[1]
        data.append(torch.cat([x, torch.zeros(1, n_part_max - n_part, 16)], dim=1).squeeze())
        lengths.append(n_part)

    # Stack the tensors along the batch dimension
    data = torch.stack(data)

    # Return the padded data, original lengths, and target labels
    return data, lengths

In [ ]:
batch_size       = 10
train_dataloader = DataLoader(train_data, batch_size=batch_size, collate_fn=custom_collate)

# loop over the dataloader to get the data in batches
i=0
for batch, original_length in train_dataloader:
    print(batch.shape, original_length)
    break

In [2]:
########## Edge Convolution Block ###########
# The root block of our DNN.
# Initialized by:
#   - d     the number of features
#   - k     number of nearest neighbours to consider in the concolution
#   - C     a list-like or an int with the number of neurons of the three linear layers
#   - aggr  the aggregation function, must be symmetric

class EdgeConv(nn.Module):
    
    def __init__(self, d, k, C, aggr=None):
        super().__init__()
        
        if type(C) == int:
            self.C = [C]*3
        else:
            self.C = C
        
        self.k = k

        if aggr is None:
            self.aggr = torch.mean
        else:
            self.aggr = aggr

        self.act = nn.ReLU()


        ### Shortcut path
        self.shortcut = nn.Sequential(
            nn.Conv1d(in_channels = d, out_channels = self.C[-1], kernel_size = 1, stride = 1),
            nn.BatchNorm1d(self.C[-1])
        )

        ### Linear section, approximation of a MLP
        self.mlp = nn.Sequential(
            nn.Conv2d(2*d, self.C[0], 1, 1),
            nn.BatchNorm2d(self.C[0]),
            nn.ReLU(),
            nn.Conv2d(self.C[0], self.C[1], 1, 1),
            nn.BatchNorm2d(self.C[1]),
            nn.ReLU(),
            nn.Conv2d(self.C[1], self.C[2], 1, 1),
            nn.BatchNorm2d(self.C[2]),
            nn.ReLU()
        )


    def kNN(self, x):
        """input: single jet data
            output: tensor with shape [B, n, k, d] where d are the features of the knn particles"""
        # expand the input tensor s.t. x_knn.shape = [B, n, n, d]
        x_knn = x.unsqueeze(1).expand(-1, x.shape[1], -1, -1)

        # calculate both delta_phi and delta_eta
        delta_phieta = x_knn[:, :, :, :2] - x_knn[:, :, :, :2].transpose(1, 2)

        # calculate distances and sort them in ascending order, keep only the indeces
        _, indeces = torch.sqrt(torch.sum(delta_phieta**2, 3)**0.5).sort(dim=2, stable=True)

        # keep the indeces of k nearest neighbours and use them to sort and cut the initial tensor
        knn = indeces[:,:,:self.k]
        x_knn = torch.gather(x_knn, 2, knn.unsqueeze(-1).expand(-1, -1, -1, x_knn.shape[-1]))

        del delta_phieta, indeces, knn, _

        return x_knn    # x_knn.shape = [B, n, k, d]

    
    def linear_aggregate(self, x):

        # accepts as input [B, d, n, k]

        # take the features of the particle and repeat them on the third axis
        p_feat = x[:, :, :, 0].unsqueeze(3).expand(-1, -1, -1, self.k)

        # now we can calculate knn features for each particle as a simple difference
        knn_feat = x - p_feat

        print('p_feat:', p_feat.shape, '\nknn_feat:', knn_feat.shape)

        pairs = torch.concat([p_feat, knn_feat], dim=1)
        del p_feat, knn_feat
        print(pairs.shape) # expected [B, 2*d, n, k]

        mlp_result = self.mlp(pairs)
        del pairs

        # aggregate
        aggr_result = self.aggr(mlp_result, dim=3)

        if type(aggr_result) is tuple:
            aggr_result = aggr_result[0]
        
        return aggr_result


    def forward(self, x):
        # x.size = [B, n, d]
        # x_knn.size = [B, n, k, d]

        x_knn = self.kNN(x).transpose(1, 3).transpose(2, 3)
        shortcut = self.shortcut(x.transpose(1, 2))
        x = self.linear_aggregate(x_knn)

        x = self.act(x + shortcut).transpose(1, 2)
        
        del x_knn, shortcut
        return x

In [4]:
class ParticleNet():
    def __init__(self, encoded_space_dim):
        super().__init__()

        # EDGE CONV PART
        self.edge_conv = nn.Sequential(
            EdgeConv(d=16, k=10, C=[64,64,64]),
            EdgeConv(d=16, k=10, C=[128,128,128]),
            EdgeConv(d=16, k=10, C=[256,256,256])) #output shape = [B,n,256]

        self.final_part = nn.Sequential(
            nn.Linear(in_features=256, out_features=256),
            nn.ReLU(),
            nn.Dropout(0.1))

        # LATENT SPACE PROJECTION
        # output size -> dimension of the latent space
        self.latent_space = nn.Sequential(
            nn.Linear(in_features = 256, out_features = encoded_space_dim),
            nn.Softmax(),
        )
    
    def forward(self, x):
        y = self.edge_conv(x)
        y = nn.AvgPool1d(kernel_size=y.shape[1], stride=1)(y.transpose(1,2))
        y = self.final_part(y)
        y= self.latent_space(y)

        return y
    

## Loss and Learing Rate

In [ ]:
### Define the loss function
loss_fn = torch.nn.MSELoss()

### Define the loss function
loss_fn = torch.nn.MSELoss()

### Define an optimizer (both for the encoder and the decoder!)
lr = 5e-4 # Learning rate

# params_to_optimize = [
#     {'params': encoder.parameters()},
#     {'params': decoder.parameters()}
# ]
# optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-5)

# # Check if the GPU is available
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# print(f'Selected device: {device}')

# # Move both the encoder and the decoder to the selected device
# encoder.to(device)
# decoder.to(device)

## Training

In [ ]:
### Training function
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()

    # abbiamo già definito l'optimizer nella cella precedente
    
    losses = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch, _ in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        
        image_batch = image_batch.to(device)
        #encode
        y_encoder_pred = encoder(image_batch)
        #decode
        y_decoder_pred = decoder(y_encoder_pred)

        loss = loss_fn(y_decoder_pred, image_batch)

        optim.zero_grad()
        loss.backward()
        optim.step()
        
        losses.append(loss.detach().cpu().numpy())
    losses = np.mean(losses)
    return losses